In [30]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings

In [71]:
train_realdata = pd.read_csv('C:/workspace/jeju/2020_jeju_creditcard/data/baselinedata.csv',index_col=0)
train_realdata.head()

C:\Users\drlim\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3


In [73]:
corona = pd.read_csv('C:/workspace/jeju/2020_jeju_creditcard/data/Corona_timeprovince.csv')
corona.head()
corona_real = corona[corona.date<="2020-03-31"]

is_thirtyone = corona_real['date'].isin(['2020-01-31','2020-02-29','2020-03-31'])
corona_real = corona_real[is_thirtyone]
corona_real

corona_real.rename(columns={'province':'CARD_SIDO_NM'},inplace=True)

corona_real = corona_real.sort_values(['CARD_SIDO_NM','date'])


corona_real

,date,time,CARD_SIDO_NM,confirmed,released,deceased
188,2020-01-31,16,Busan,0,0,0
681,2020-02-29,16,Busan,80,0,0
1208,2020-03-31,0,Busan,119,87,3
197,2020-01-31,16,Chungcheongbuk-do,0,0,0
690,2020-02-29,16,Chungcheongbuk-do,10,0,0
1217,2020-03-31,0,Chungcheongbuk-do,44,22,0
198,2020-01-31,16,Chungcheongnam-do,0,0,0
691,2020-02-29,16,Chungcheongnam-do,55,0,0
1218,2020-03-31,0,Chungcheongnam-do,128,90,0
189,2020-01-31,16,Daegu,0,0,0


In [74]:
n = corona_real.CARD_SIDO_NM.nunique()

for i in range(1,n):
    for j in range(3,5):
        corona_real.iloc[3*i-2,j] = corona_real.iloc[3*i-2,j] - corona_real.iloc[3*i -3,j]
        corona_real.iloc[3*i-1,j] = corona_real.iloc[3*i-1,j] - corona_real.iloc[3*i -2,j]
corona_real

,date,time,CARD_SIDO_NM,confirmed,released,deceased
188,2020-01-31,16,Busan,0,0,0
681,2020-02-29,16,Busan,80,0,0
1208,2020-03-31,0,Busan,39,87,3
197,2020-01-31,16,Chungcheongbuk-do,0,0,0
690,2020-02-29,16,Chungcheongbuk-do,10,0,0
1217,2020-03-31,0,Chungcheongbuk-do,34,22,0
198,2020-01-31,16,Chungcheongnam-do,0,0,0
691,2020-02-29,16,Chungcheongnam-do,55,0,0
1218,2020-03-31,0,Chungcheongnam-do,73,90,0
189,2020-01-31,16,Daegu,0,0,0


In [75]:
corona_real.replace(['Seoul','Gyeonggi-do','Gyeongsangnam-do','Daegu','Incheon','Chungcheongnam_do','Gyeongsangbuk-do','Busan','Daejeon','Gwangju','Chungcheongbuk-do','Jeollabuk-do','Ulsan','Jeollanam-do','Sejong','Gangwon-do','Jeju-do','Chungcheongnam-do'],['서울','경기','경남','대구','인천','충남','경북','부산','대전','광주','충북','전북','울산','전남','세종','강원','제주','충남'],inplace=True)

In [76]:
corona_real['year'] = corona_real['date'].astype(str).str[:4]
corona_real['month'] = corona_real['date'].astype(str).str[5:7]
corona_real['month'] = corona_real['month'].astype(int)
corona_real['month'] = corona_real['month'].astype(str)

corona_real = corona_real.drop(columns=['date','time'])


In [77]:
train_realdata.dtypes 

CARD_SIDO_NM    object
STD_CLSS_NM     object
HOM_SIDO_NM     object
AGE             object
SEX_CTGO_CD      int64
FLC              int64
year             int64
month            int64
CSTMR_CNT        int64
AMT              int64
CNT              int64
dtype: object

In [78]:
corona_real.dtypes

CARD_SIDO_NM    object
confirmed        int64
released         int64
deceased         int64
year            object
month           object
dtype: object

In [79]:
corona_real.year = corona_real.year.astype(int)
corona_real.month = corona_real.month.astype(int)

In [80]:
merge = pd.merge(train_realdata, corona_real, left_on = ['CARD_SIDO_NM','year','month'], right_on = ['CARD_SIDO_NM','year','month'], how='left') 
merge = merge.fillna(0)

## 중요! 누적인데 diff를 해야 한다! 

1월 1일 ~ 3월 31일까지 증가만 하는 것이 의심이 들었는데 알고 보니 누적 확진자 / released/ 사망자 수였다

- 그러므로 1월 31일이 1월 전체이며 2월 29일이 1~2월 전체이며 3월 31일이 1~3월 전체이다.

- 나는 1/31, 2/29, 3/31로 하였다

- 그러므로 마지막 단계는 2월 - 1월, 3월 - 2월 하기다!

In [81]:
merge

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT,confirmed,released,deceased
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4,0.000,0.000,0.000
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3,0.000,0.000,0.000
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3,0.000,0.000,0.000
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3,0.000,0.000,0.000
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3,0.000,0.000,0.000
5,강원,건강보조식품 소매업,강원,20s,1,1,2019,11,13,843100,16,0.000,0.000,0.000
6,강원,건강보조식품 소매업,강원,20s,1,1,2020,1,6,168500,6,0.000,0.000,0.000
7,강원,건강보조식품 소매업,강원,20s,1,1,2020,3,3,427510,2,29.000,23.000,1.000
8,강원,건강보조식품 소매업,강원,20s,2,1,2019,2,4,912000,5,0.000,0.000,0.000
9,강원,건강보조식품 소매업,강원,20s,2,1,2019,3,12,631000,13,0.000,0.000,0.000
